In [1]:
import json
import torch

In [2]:
def find_word(sent_list,temp_list):      #一个list里找另一个list的函数，主要是针对arg1和arg2的span
    result=[]
    temp_len=len(temp_list)
    for i in range(len(sent_list)-temp_len):
        if sent_list[i:i+temp_len]==temp_list:
            result.append(list(range(i,i+temp_len)))      #本来是简单的append[i,i+temp_len]，但是为了方便后面的find_span，就改了
            break
    if result==[]:
        result=[[]]
    return result[0]

def remove(a_list,a):                    #list.remove不好用，只能自己创建一个函数来去除多余的''
    new_list=[]
    for i in a_list:
        if i==a:
            continue
        new_list.append(i)
    return new_list

In [3]:
def find_span(sent_list,temp_list):                        #给定句子的token_list和arg的token_list然后进行递归匹配子块
    result=find_word(sent_list,temp_list)
    if result==[]:                                         #没匹配上，说明要么是两个词块拼一起，要么就是有句中不存在的词                  
        if temp_list==[]:                                  #比如rel里只有is且is还不在句子里
            return []
        temp_result=[]
        for i in range(0,2):
            new_temp_list=temp_list[i:len(temp_list)-1+i]    #取小的子块儿
            temp_result.append(find_span(sent_list,new_temp_list))                    #递归
        result_len=[len(j) for j in temp_result]
        position=result_len.index(max(result_len))
        result=temp_result[position]
    return result
#测试完毕，终于搞定了，很NB！

In [4]:
def deal_tag(sent_list,tag):
    new_sent=sent_list+['is','have','from']
    a1,a2,rel=[],[],[]
    for i in range(len(tag)):
        if tag[i]==1 or tag[i]==2:
            a1.append(sent_list[i])
        if tag[i]==3 or tag[i]==4:
            a2.append(sent_list[i])
        if tag[i]==5:
            rel.append(new_sent[i])
    return ' '.join(a1),' '.join(a2),' '.join(rel)

In [5]:
#筛选一下三元组，保留arg2存在且不重复的三元组
def filter1(gold_data):
    temp=[]
    for gold in gold_data:
        data=gold.split('\t')
        if len(data)<4:
            continue
        if len(data)>4:
            new_gold='\t'.join(data[:4])+'\n'
            if new_gold not in temp:
                temp.append(new_gold)
        if len(data)==4 and gold not in temp and data[3]!='\n':    #arg2即使没有可能还会是\n
            temp.append(gold)
    print(len(temp),len(gold_data))
    return temp

In [8]:
def deal_carb(temp,train=False):
    sent=temp[0].split('\t')[0]
    index,result,arg1_list,arg2_list,tags_list,arg1_str,arg2_str,rel_str=0,[],[],[],[],[],[],[]
    n1,n2,n3=0,0,0
    for temp_data in temp:
        data=temp_data.split('\t')      #data[0]是句子，data[1]是rel，data[2]是arg1，data[3]是arg2
        if not data[0]==sent:
            if arg1_list!=[] and arg2_list!=[] and tags_list!=[]:
                if train:               #train实在太多了，写入str的话会非常麻烦！
                    feature={'sent':new_sent_list,'a1_tags':arg1_list,'a2_tags':arg2_list,'sent_id':index,'tags':tags_list}
                else:
                    feature={'sent':new_sent_list,'a1_tags':arg1_list,'a2_tags':arg2_list,'sent_id':index,'tags':tags_list,
                        'a1':arg1_str,'a2':arg2_str,'rel':rel_str}
                result.append(feature)
            sent=data[0]
            index+=1
            arg1_list,arg2_list,tags_list,arg1_str,arg2_str,rel_str=[],[],[],[],[],[]
        
        sent_list=data[0].split(' ')
        new_sent_list=remove(sent_list,'')          #有的句子最后的字符是''，tokenizer处理不了
        arg1=data[2].split(' ')
        arg2=data[3][:-1].split(' ')                #'/n'算一个字符
        rel=data[1].split(' ')
        new_arg1=remove(arg1,'')
        new_arg2=remove(arg2,'')
        new_rel=remove(rel,'')                       #这个数据不太行，这里这个rel的处理还得研究，包括这个arg也是，就离谱
        if len(new_arg1)>8 or len(new_arg2)>8 or len(new_rel)>8: #span长度过长，但一般carb会好很多
            n1+=1
            continue
        
        if train:                                    #主要是train数据太多，所以在arg这里就不做子块筛选了，只做原本的span筛选
            arg1_tag=find_word(new_sent_list,new_arg1)
            arg2_tag=find_word(new_sent_list,new_arg2)
        else:
            arg1_tag=find_span(new_sent_list,new_arg1)
            arg2_tag=find_span(new_sent_list,new_arg2)
        if arg1_tag==[] or arg2_tag==[]:                          #理论上如果用find_span,n2应该是0
            n2+=1
            continue
        rel_tag=find_span(new_sent_list,new_rel)
        #print(new_arg1,arg1_tag,new_arg2,arg2_tag)
 
        tag=torch.zeros(len(new_sent_list)+3)
        tag[arg1_tag[0]]=1
        tag[arg1_tag[0]+1:arg1_tag[-1]+1]=2
        tag[arg2_tag[0]]=3
        tag[arg2_tag[0]+1:arg2_tag[-1]+1]=4
        if rel_tag!=[]:
            tag[rel_tag[0]:rel_tag[-1]+1]=5
        else:
            if 'is' in rel or 'was' in rel:                       #分别代表is,of,from三个词凭空出现在rel里
                tag[-3]=5
            if 'of' in rel:
                tag[-2]=5
            if 'from' in rel:
                tag[-3]=5
            if 5 not in tag:                                      #这些准备都做完了还是没有rel，那可以拜拜了
                n3+=1
                continue
                
        arg1_list.append(arg1_tag)
        arg2_list.append(arg2_tag)
        tags_list.append(tag.tolist())

        if train:
            continue
        a1,a2,re=deal_tag(new_sent_list,tag)                          #根据处理后的tag来得到a1,a2和rel，为下一步清洗做铺垫，因为取子块会有相似数据
        arg1_str.append(a1)
        arg2_str.append(a2)
        rel_str.append(re)
        
    print(n1,n2,n3,len(temp),len(result))
    return result
    #这个函数意思是在test里进行条件筛选，筛选是，arg1和arg2必须是连续的span，还有rel词如果多了隐藏词，只能是is,of,from

In [9]:
with open('../openie6-master/carb/data/gold/dev.tsv','r') as f:
    gold_data=f.readlines()
temp=filter1(gold_data)
new_data=deal_carb(temp)
with open('datasets/CaRB/dev.json','w') as f:
    json.dump(new_data,f)
#完美！

2455 2548
450 17 185 2455 598


In [30]:
with open('../openie6-master/imojie/data/train/4cr_qpbo_extractions.tsv','r') as f:
    temp_data=f.readlines()
#这里不如把这个数据变成gold_data的形式来试试
#print(len(temp_data),temp_data[5])
#这里设置了粗暴的筛选，因为数据太多了，首先，span只要不挨着，就算了，然后长度不能大于8，rel还是加了is,from,have
filter_data=[]
for data in temp_data:
    data1=data.replace('<arg1>','')
    data2=data1.replace('</arg1> <rel>','|')
    data3=data2.replace('</arg2>','')
    data4=data3.replace('</rel> <arg2>','|')
    new_data=data4.split('\t')
    #print(data,new_data)
    arg1=new_data[1].split('|')[0][1:-1]
    arg2=new_data[1].split('|')[2]
    rel=new_data[1].split('|')[1]
    temp_result=new_data[0]+'\t'+rel+'\t'+arg1+'\t'+arg2+'\n'
    filter_data.append(temp_result)
#主要是把数据变成carb的形式好实用下面的函数

In [31]:
%time new_data=deal_carb(filter_data,True)    #只剩下10W数据哈哈哈哈哈哈或
with open('datasets/CaRB/train.json','w') as f:    #好家伙，写数据写了2个小时！？397个G！？
   json.dump(new_data,f)

83511 29330 9 215356 66196
CPU times: user 5.28 s, sys: 200 ms, total: 5.48 s
Wall time: 5.48 s


In [ ]:
#这里设计一个相似度函数来筛选一些相似的数据，以tag为基准就可以，如果两个tag的相似度在多少以上就可以，这个可以下一步看，看看具体能筛选多少

In [12]:
def tag_simliar(tag1,tag2):
    x=(tag1==tag2).float()
    similiar=torch.sum(x)/len(tag1)
    return similiar

In [20]:
def delettag(tag):   
    result=[]
    for i in range(len(tag)-1):
        for j in range(len(tag[i+1:])):
            new_j=j+i+1
            simliar=tag_simliar(tag[i],tag[new_j])     #比较任意两个tag的相似性
            if simliar>0.9:                                  
                tag1_len=torch.sum((tag[i]>0).float())
                tag2_len=torch.sum((tag[new_j]>0).float())
                if tag1_len<tag2_len and i not in result: 
                    result.append(i)
                if not tag1_len<tag2_len and new_j not in result:
                    result.append(new_j)
    return result
#返回要删除的句子的序号，可以试一下

In [14]:
def remove_elements(lista,index_list):
    result=[]
    for i,item in enumerate(lista):
        if i not in index_list:
            result.append(item)
    return result
#lista=['a','b','c','d','e','a','a']
#listb=[1,4,6]

In [15]:
def get_new(tempdata,deletlist,train=False):
    sent=tempdata['sent']
    sent_id=tempdata['sent_id']
    new_a1=remove_elements(tempdata['a1_tags'],deletlist)
    new_a2=remove_elements(tempdata['a2_tags'],deletlist)
    new_tag=remove_elements(tempdata['tags'],deletlist)
    if train:
        feature={'sent':sent,'a1_tags':new_a1,'a2_tags':new_a2,'sent_id':sent_id,'tags':new_tag}
    else:
        feature={'sent':sent,'a1_tags':new_a1,'a2_tags':new_a2,'sent_id':sent_id,'tags':new_tag,
                'a1':tempdata['a1'],'a2':tempdata['a2'],'rel':tempdata['rel']}
    return feature

In [32]:
with open('datasets/CaRB/train.json','r') as f:
    test_data=json.load(f)

In [33]:
#这里就这么简单粗暴的删除一下吧，然后看看效果，先存个新train、dev、test
new_data,test,delete_num=[],[],0
for data in test_data:
    tag=torch.tensor(data['tags'])
    test.append(delettag(tag))
#test里是要删除的tag的序列，有可能为空
for i in test:
    if not i==[]:
        delete_num+=len(i)
print(delete_num)
for i in range(len(test)):
    if test[i]==[]:
        new_data.append(test_data[i])
    else:
        new_feature=get_new(test_data[i],test[i],True)
        new_data.append(new_feature)
with open('datasets/CaRB/train_no_similiar.json','w') as f:
    json.dump(new_data,f)
#train数据里10W个数据，才2400个相似的，所以训练看来无所谓了，test和dev都是接近2000的数据，test里有225个,dev里面191个相似的，很有趣

2390


In [34]:
def write_tsv(data,file_name):
    with open(file_name,'w') as f:
        for temp_data in data:
            for i in range(len(temp_data['tags'])):
                f.write(' '.join(temp_data['sent'])+'\t'+temp_data['rel'][i]+'\t'+temp_data['a1'][i]+'\t'+temp_data['a2'][i]+'\n')

In [36]:
#把test和dev的处理后的数据写成tsv看看吧
with open('datasets/CaRB/dev.json','r') as f:
    dev_data=json.load(f)
with open('datasets/CaRB/test.json','r') as f:
    test_data=json.load(f)
with open('datasets/CaRB/dev_no_similiar.json','r') as f:
    new_dev_data=json.load(f)
with open('datasets/CaRB/test_no_similiar.json','r') as f:
    new_test_data=json.load(f)

write_tsv(dev_data,'../openie6-master/MyORE_test/gold/dev_filter.tsv')
write_tsv(test_data,'../openie6-master/MyORE_test/gold/test_filter.tsv')
write_tsv(new_dev_data,'../openie6-master/MyORE_test/gold/dev_no_similiar.tsv')
write_tsv(new_test_data,'../openie6-master/MyORE_test/gold/test_no_simililar.tsv')
#原本的dev,2541,筛选后,1906,相似度筛选后，1712
#原本的test,2691,筛选后,1892,相似度筛选后，1667
